In [10]:
%load_ext autoreload
%autoreload 2

import pickle
import sys
import numpy as np
from macaque_reaching_helpers import *
import MARBLE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
"""fitting model for each day + pca embedding"""
    
data_file = "data/rate_data_20ms.pkl"
metadata_file = "data/trial_ids.pkl"

rates, trial_ids = load_data(data_file, metadata_file)

# defining the set of conditions
conditions = ["DownLeft", "Left", "UpLeft", "Up", "UpRight", "Right", "DownRight"]

# list of days
days = rates.keys()

# define some parameters
pca_n = 5
filter_data = True

# storing all distance matrices
embeddings = []
distance_matrices = []
times = [] # to store the time point of each node in the trajectory
all_condition_labels = [] # to store the condition label for each node
all_trial_ids = [] # trial ids for each node
all_sampled_ids = [] # to store all the nodes sampled by marble

# loop over each day
day = 5

# first stack all trials from that day together and fit pca
print(day)
#pca = fit_pca(rates, day, conditions, filter_data=filter_data, pca_n=pca_n)
pos, vel, timepoints, condition_labels, trial_indexes = format_data(rates, 
                                                                    trial_ids,
                                                                    day, 
                                                                    conditions, 
                                                                    #pca=pca,
                                                                    filter_data=filter_data)


File ‘data/rate_data_20ms.pkl’ already there; not retrieving.
File ‘data/trial_ids.pkl’ already there; not retrieving.


5


In [12]:
# construct data for marble
data = MARBLE.construct_dataset(
    anchor=pos,
    vector=vel,
    k=30,
    spacing=0.05,
    delta=1.5,
    pca_dim=pca_n,
    #metric='cityblock'
)

params = {
    "epochs": 120,  # optimisation epochs
    "order": 2,  # order of derivatives
    "hidden_channels": 100,  # number of internal dimensions in MLP
    "out_channels": 3, # or 3 for Fig3
    "inner_product_features": False,
    "vec_norm": False,
    "diffusion": True,
}

model = MARBLE.net(data, params=params)

model.fit(data, outdir="data/session_{}_20ms".format(day))
data = model.transform(data)

n_clusters = 50
data = MARBLE.distribution_distances(data, n_clusters=n_clusters)

embeddings.append(data.emb)
distance_matrices.append(data.dist)
times.append(np.hstack(timepoints))
all_condition_labels.append(data.y)
all_trial_ids.append(np.hstack(trial_indexes))
all_sampled_ids.append(data.sample_ind)

# save over after each session (incase computations crash)
with open("data/marble_embeddings_20ms_out3.pkl", "wb") as handle:
    pickle.dump(
        [
            distance_matrices,
            embeddings,
            times,
            all_condition_labels,
            all_trial_ids,
            all_sampled_ids,
        ],
        handle,
        protocol=pickle.HIGHEST_PROTOCOL,
    )


---- Embedding dimension: 5
---- Signal dimension: 5
---- Computing kernels ... 
---- Settings: 

epochs : 120
order : 2
hidden_channels : [100]
out_channels : 3
inner_product_features : False
vec_norm : False
diffusion : True
batch_size : 64
lr : 0.01
momentum : 0.9
frac_sampled_nb : -1
include_positions : False
include_self : True
dropout : 0.0
bias : True
batch_norm : batch_norm
emb_norm : False
seed : 0
dim_signal : 5
dim_emb : 5
n_sampled_nb : -1

---- Number of features to pass to the MLP:  155
---- Total number of parameters:  16104

Using device cuda:0

---- Training network ...

---- Timestamp: 20240424-175005

Epoch: 0, Training loss: 1.335511, Validation loss: 1.3720, lr: 0.0100 *
Epoch: 1, Training loss: 1.278198, Validation loss: 1.4691, lr: 0.0100
Epoch: 2, Training loss: 1.230770, Validation loss: 1.2851, lr: 0.0100 *

KeyboardInterrupt: 